In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

In [ ]:
data = pd.read_csv('./Data/CS_dataset.csv' )  # nrows = 1000000

In [ ]:
#If we want to reduce the size of the data:
data = data.sample(frac = 0.05)

In [ ]:
 data = data.dropna() #drop lines containing NaN values

In [ ]:
#data.isna().any()

In [ ]:
# data[data.index == 79695] #Corse

In [ ]:
del data['specialty_subgroup'] #deletes the unuseful column specialty_subgroup
del data['id'] #Delete ID which is useless

In [ ]:

# set(list(data['zipcode'])) #list of zip codes
#len(data[data['no_show']== 1]) #percentage of no_shows in the sample data

In [ ]:
new_patient_class = list(data['zipcode'].value_counts().keys())
#len(new_patient_class)

### get_period_day bucketizes the time in 4 periods of the day


In [ ]:

def get_period_day(time):
    if 7 < int(time[0:2]) < 11:
        return 0 #morning
    if 11 <= int(time[0:2]) <= 14:
        return 1 #lunch
    if 14 < int(time[0:2]) <= 19:
        return 2 #afternoon
    else:
        return 3 #night

In [ ]:
data['start'] = data['start'].apply(lambda time: get_period_day(time)) 
data['created'] = data['created'].apply(lambda time: get_period_day(time))
    

In [ ]:
#create dictionary for 
zip_region = pd.read_csv('./Data/zip-nom-region.csv', delimiter = ';') #data with the mapping from zip code to region namezip_region.drop_duplicates(inplace = True)
zip_region.drop_duplicates(inplace = True)

In [ ]:
zip_region.index = zip_region['Code Dept']
del zip_region['Code Dept']
dict_zip_region = zip_region.to_dict()['Nom Region']

def return_region(depto_code):
    return dict_zip_region[str(depto_code)]


In [ ]:
dict_zip_region['98'] = 'Monaco' #????
dict_zip_region['97'] = 'La Reunion' #???
dict_zip_region['20'] = 'Corse'

In [ ]:
#put LINGOLSHEIM in the same zip department as Strasbourg (67)

def get_department(zip_code):
    if str(zip_code).split()[0].isdigit(): #'68125 '.split()[0].isdigit()
        #print('return ', int(str(zip_code)[0:2]) )
        return int(str(zip_code)[0:2])
    if zip_code == 'LINGOLSHEIM':
        #print('LINGO 67')
        return 67
    if zip_code == 'L-4048':
        return 64 #Aquitaine
    else:
        print('data type: {} , zip code {}: '.format(type(zip_code) , zip_code))
        return np.nan

In [ ]:
data['zipcode'] = data['zipcode'].apply(lambda code: get_department(code))

In [ ]:
#data = data.dropna() #hopefully not necessary 

In [ ]:
#create empy column for regions
data['region'] = ""


In [ ]:
def get_region_name(zip_code):
    #dept_code = get_department(zip_code)
    #print(zip_code)
    region = zip_region[zip_region['Code Dept'] == '6']['Nom Region'].item()
    #print("zipcode: {} , region: {}: ".format(zip_code , region))
    return region  #list(zip_region[zip_region['Code Dept'] == '6']['Nom Region'])[0]
    

In [ ]:
data['region'] = data['zipcode'].apply(lambda code: return_region(code))

## Let's encode the columns that are not numeric

In [ ]:
new_patient_dummy = pd.get_dummies(data['new_patient'], prefix=['new_patient']).copy()

In [ ]:
### The missing data isn't taken into account (NaN). Should we add it anyways ?
is_relative_dummy =  pd.get_dummies(data['is_relative'], prefix=['is_relative']).copy()

In [ ]:
booking_device_dummy = pd.get_dummies(data['booking_device'], prefix=['booking_device']).copy()

In [ ]:
organization_type_dummy = pd.get_dummies(data['organization_type'], prefix=['organization_type']).copy()

In [ ]:
patient_age_group_dummy = pd.get_dummies(data['patient_age_group'], prefix=['patient_age_group']).copy()

In [ ]:
visit_motive_name_dummy = pd.get_dummies(data['visit_motive_name'], prefix=['visit_motive_name']).copy()

In [ ]:
month_dummy =  pd.get_dummies(data['month'], prefix=['month']).copy()

In [ ]:
day_dummy = pd.get_dummies(data['day'], prefix=['day']).copy()

In [ ]:
start_dummy = pd.get_dummies(data['start'], prefix=['start']).copy()

In [ ]:
created_dummy = pd.get_dummies(data['created'], prefix=['created']).copy()

In [ ]:
region_dummy = pd.get_dummies(data['region'], prefix=['region']).copy()

Now we drop these columns

In [ ]:
data.drop(['new_patient', 'is_relative', 'booking_device', 'organization_type', 'patient_age_group', 'visit_motive_name', 'month', 'day', 'start', 'created', 'region'], axis=1, inplace = True)

And we concatenate them all

In [ ]:
bucketized_df = pd.concat([data, patient_age_group_dummy, visit_motive_name_dummy,organization_type_dummy,booking_device_dummy, is_relative_dummy,new_patient_dummy, month_dummy, day_dummy, start_dummy,created_dummy, region_dummy], axis=1)


And save them to a CSV file

In [ ]:
bucketized_df.to_csv('encoded_data_5%.csv')